In [1]:
import pandas as pd

file_name = "df_vector.pkl"

In [2]:
df_vector = pd.read_pickle(file_name)
df_vector.head()

,Theme,FeatureVector,WordVector
0,Mathematics,"[0.04838709677419355, 0.03225806451612903, 0.0...","[0, zero, number, numerical, digit, use, repre..."
Abiogenesis,Science,"[0.003003003003003003, 0.003003003003003003, 0...","[biology, abiogenesis, ‘, ’, greek, bios, ‘, l..."
Abortion,Society and social sciences,"[0.01282051282051282, 0.003205128205128205, 0....","[abortion, termination, pregnancy, removal, ex..."
Abraham Lincoln,People,"[0.0, 0.0030581039755351682, 0.006116207951070...","[abraham, lincoln, link, ən, february, 12, 180..."
Abraham,People,"[0.0, 0.005681818181818182, 0.0056818181818181...","[abraham, originally, abram, common, hebrew, p..."


# Test Train Split For Vectors
No need to standerdize the data as the vectors are standardized

In [3]:
x_vector = df_vector.FeatureVector.copy()
y_vector = df_vector.Theme.copy()
y_vector.head()

0                                  Mathematics
Abiogenesis                            Science
Abortion           Society and social sciences
Abraham Lincoln                         People
Abraham                                 People
Name: Theme, dtype: object

In [4]:
from sklearn.model_selection import train_test_split

x_vec_train, x_vec_test, y_vec_train, y_vec_test = train_test_split(x_vector, y_vector, test_size=0.2, random_state=5)

In [5]:
x_vec_test.head()

Tea                                                  [0.0, 0.006944444444444444, 0.0069444444444444...
Hippocrates                                          [0.01680672268907563, 0.0, 0.01680672268907563...
Medication                                           [0.024539877300613498, 0.006134969325153374, 0...
International Red Cross and Red Crescent Movement    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Black Death                                          [0.0, 0.0, 0.008620689655172414, 0.00862068965...
Name: FeatureVector, dtype: object

In [6]:
y_vec_test.head()

Tea                                                                 Everyday life
Hippocrates                                                                People
Medication                                           Health, medicine and disease
International Red Cross and Red Crescent Movement     Society and social sciences
Black Death                                                               History
Name: Theme, dtype: object

Check for NaN

In [7]:
tmp = [x_vec_train, x_vec_test, y_vec_train, y_vec_test]

for e in tmp:
    print(e.isnull().values.any())

False
False
False
False


In [8]:
x_vec_train = x_vec_train.to_numpy()

# K-Means


In [9]:
import numpy as np

x_vec_train = np.stack(x_vec_train)

In [10]:
from sklearn.cluster import KMeans

k = 11
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(x_vec_train)

In [11]:
y_pred

array([ 7, 10,  6,  7, 10,  7,  7, 10,  2, 10, 10,  2, 10, 10, 10,  2,  1,
        2, 10,  2,  2,  7,  2, 10,  3, 10, 10,  7, 10,  2,  7, 10, 10,  2,
        2, 10,  7, 10, 10,  2,  0, 10, 10,  8, 10, 10, 10, 10,  2, 10, 10,
       10,  7,  2, 10, 10, 10, 10, 10, 10,  2,  2, 10,  2,  2,  7,  7,  7,
        2, 10,  3, 10, 10,  7, 10,  7, 10,  7,  2, 10, 10,  7,  7, 10,  7,
        2,  7, 10,  7, 10,  2, 10, 10,  3,  7, 10, 10, 10, 10,  8,  0, 10,
        2, 10,  1,  7, 10,  3, 10,  1,  7,  7, 10,  2,  7,  7, 10, 10,  2,
        7,  2, 10,  7,  2,  2,  3, 10, 10, 10, 10,  7,  7, 10, 10,  8, 10,
       10,  7,  7,  2,  7, 10,  2, 10, 10, 10,  2, 10,  7, 10, 10, 10, 10,
       10, 10, 10,  7,  2, 10,  1,  7,  6, 10, 10, 10,  7,  7,  7, 10,  7,
       10, 10, 10, 10, 10,  7, 10,  7,  7,  7, 10, 10,  7, 10, 10, 10, 10,
       10, 10, 10,  7,  7,  7,  7,  8,  8,  9, 10,  6,  8, 10,  2,  2, 10,
        7,  7, 10, 10, 10, 10,  4, 10,  7,  7,  2, 10, 10,  6, 10, 10, 10,
       10,  2, 10,  2, 10